In [4]:
import bql
from ipywidgets import Button, VBox, HTML, Output

from IPython.display import display, clear_output
import pickle
import importlib

import gui.fund_charts
import gui.table_boxplot
import gui.gui_utils_ryo
import gui.fund_filters


bq = bql.Service()

In [5]:
class FundCalculations():
    def __init__(self, bq):
        """ This class contains all the calculations to be performed on 
        the selected fund universe. It can list all available functions, 
        and it can return a list of BQL data objects for the selected 
        list of calculations. 
        
        """ 
        self.__bq = bq
        self._calculations = {}
        self._calculation_orders = {}
        self.__map_calculations()
        self.__map_calculation_order()
        
    def __map_calculations(self):
        """ Start a dictionary where key is mapped to the function that creates the
        bql data object. So we can list all available calcs and feed them to the gui. 
        Later from the gui we can choose which calcs we want, and call with selected parameters.
        
        To add a new calculation, we need to add 3 entries, one in __map_calculations(),
        one in __map_calculation_order(), and we need a function that takes the selected settings,
        and returns the bql field object.
        """
        
        self._calculations["Total_Return"] = self.__calc_tr
        self._calculations["Total_Return_Annual"] = self.__calc_tr_annualised
        self._calculations["Volatility"] = self.__calc_vol
        self._calculations["Alpha"] = self.__calc_alpha
        self._calculations["Beta"] = self.__calc_beta
        self._calculations["Downside_Volatility"] = self.__calc_downside_vol
        self._calculations["Sharpe_Ratio"] = self.__calc_sharpe_ratio
        self._calculations["Max_Drawdown"] = self.__calc_max_drawdown
        self._calculations["Sortino_Ratio"] = self.__calc_sortino_ratio
        self._calculations["Tracking_Error"] = self.__calc_tracking_error
        self._calculations["Calmar_Ratio"] = self.__calc_calmar_ratio
        self._calculations["Excess_Return"] = self.__calc_excess_return
        self._calculations["Gain_To_Loss"] = self.__calc_gain_to_loss_ratio
        self._calculations["Information_Ratio"] = self.__calc_information_ratio
        self._calculations["Jensen_Alpha"] = self.__calc_jensen_alpha
        
        
    def __map_calculation_order(self):
        """ We map for each calculation if higher, or lower is better"""
        self._calculation_orders["Total_Return"] = 'higher'
        self._calculation_orders["Total_Return_Annual"] = 'higher'
        self._calculation_orders["Volatility"] = 'lower'
        self._calculation_orders["Alpha"] = 'higher'
        self._calculation_orders["Beta"] = 'higher'
        self._calculation_orders["Downside_Volatility"] = 'lower'
        self._calculation_orders["Sharpe_Ratio"] = 'higher'
        self._calculation_orders["Max_Drawdown"] = 'lower'
        self._calculation_orders["Sortino_Ratio"] = 'higher'
        self._calculation_orders["Tracking_Error"] = 'lower'
        self._calculation_orders["Calmar_Ratio"] = 'higher'
        self._calculation_orders["Excess_Return"] = 'higher'
        self._calculation_orders["Gain_To_Loss"] = 'higher'
        self._calculation_orders["Information_Ratio"] = 'higher'
        self._calculation_orders["Jensen_Alpha"] = 'higher'
        
    def get_rank_mode(self, calc_name):
        return self._calculation_orders[calc_name]
        
    def __make_dt_rng(self, st_dt, ed_dt):
        return self.__bq.func.range(st_dt, ed_dt)
    
    def get_all_calc_names(self):
        available_calcs = list(self._calculations.keys())
        available_calcs.sort()
        return available_calcs
        
    def get_flds(self, requested_calcs, st_dt, ed_dt, fx, per, base, bench, annual_factor, annualize, rf_id):
        """
        st_dt / ed_dt are date string in yyyy-mm-dd format, or datetime objects
        annual_factor: defaults are D : 252; W: 52; M:12, but could set ie D to 260 if desired
        base is "NAV_IF_AVAILABLE", "PRICE_IF_AVAILABLE".
        annualize True or False, whether you want the number to be annualised
        rf_id the riskfree ticker for some of the calcs
        the rest should be self explanatory
        """
        # We define all the paramteres we want to work with, and then send them when 
        # initialising the data objects. Each function will then pick the relevant values. 
        dt_rng = bq.func.range(st_dt, ed_dt)
        params = self.__pack_parameters(dt_rng, fx, per, base, bench, annual_factor, annualize, rf_id)
        # reset 
        flds = {}
        for calc in requested_calcs:
            # self.__calculations contains a reference to the function, so we pass all params,
            # the function will then pick the required params, so its easier to write. 
            flds[calc] = self._calculations[calc]( **params)
        return flds
    
    def get_hist_flds(self, requested_calc, st_offset, fx, per, base, bench, annual_factor, annualize, rf_id, roll_st_dt, roll_end_dt, roll_per):
        f = self.__bq.func
        # st_dt will be number of days in -200d format.
        dt_rng = f.range(st_offset, '0d')
        # get parameters, turn to dictionary so we can pass it as **params to dictionary of functions that returns the data object
        params = self.__pack_parameters(dt_rng, fx, per, base, bench, annual_factor, annualize, rf_id)
        # get data field
        fld = self._calculations[requested_calc]( **params)
        # get hist series of prev 3
        hist_dt_rng = bq.func.range(roll_st_dt, roll_end_dt, frq=roll_per)
        # generate history
        fld_hist = f.rolling(fld, iterationdates = hist_dt_rng)
        # get hist univ avg
        avg_fld = f.avg(f.group(fld_hist, fld_hist['ITERATION_DATE']))
        # get hist univ std
        std_fld = f.std(f.group(fld_hist, fld_hist['ITERATION_DATE']))
        
        return fld_hist, avg_fld, std_fld
    
    def __pack_parameters(self, dt_rng, fx, per, base, bench, annual_factor, annualize, rf_id):
        params = {}
        params['dt_rng'] = dt_rng
        params['fx'] = fx
        params['per'] = per
        params['base'] = base
        params['bench'] = bench
        params['annualize'] = annualize
        params['rf_id'] = rf_id
        params['annual_factor'] = annual_factor 
        return params
    
    def __get_base(self, **params):
        if params['base'] == "PRICE_IF_AVAILABLE" :
            return "PCT_CHG_PX" 
        else:
            return "PCT_CHG_NAV" 
            
    def __sort_cols(self):
        """ Take columns, start with ID and Name, order the rest alphabetically."""
        cols = self.__data
        first_cols = ['ID', 'Name']
        cols.remove('ID')
        cols.remove('Name')
        cols.sort()
        sorted_cols = first_cols + cols
        self.__data = self.__data[sorted_cols]
        
        
    #**********************************************************************
    #
    #     calculation functions
    #
    #**********************************************************************
    
    def __calc_tr(self, **params):
        tr = self.__bq.data.TOTAL_RETURN(calc_interval = params['dt_rng'], 
                                      currency = params['fx'],
                                      TR_Base = params['base'])		
        return tr
    
    def __calc_tr_annualised(self, **params):
        tr_y = self.__bq.data.ANN_TOTAL_RETURN(calc_interval = params['dt_rng'], 
                                            currency = params['fx'],
                                            TR_Base = params['base'])		
        return tr_y
        
    def __calc_vol(self,**params):
        vol = self.__bq.data.VOLATILITY(calc_interval = params['dt_rng'], 
                                     per=params['per'],
                                     currency = params['fx'],
                                     tr_base = params['base'])		
        return vol
        
    def __calc_downside_vol(self, **params):
        downside_vol = self.__bq.data.DOWNSIDE_VOLATILITY(calc_interval = params['dt_rng'], 
                                                           per=params['per'],
                                                           tr_base = params['base'],
                                                           currency = params['fx'])
        if params['annual_factor'] != "":
            downside_vol = downside_vol.with_additional_parameters(ann_factor = params['annual_factor'])
        return downside_vol
    
    def __calc_sharpe_ratio(self, **params):
        sharpe = self.__bq.data.SHARPE_RATIO(calc_interval = params['dt_rng'], 
                                             per = params['per'],
                                             tr_base = params['base'],
                                             currency = params['fx'],
                                             risk_free_ticker = [params['rf_id']])
        if params['annual_factor'] != "":
            sharpe = sharpe.with_additional_parameters(ann_factor = params['annual_factor'])

        return sharpe
    
    def __calc_max_drawdown(self, **params):
        max_dd = self.__bq.data.Max_Drawdown(calc_interval = params['dt_rng'], 
                                      per=params['per'],
                                      currency = params['fx'],
                                      tr_base = params['base'])
        return max_dd
        
    def __calc_sortino_ratio(self, **params):
        sortino = self.__bq.data.SORTINO_RATIO(calc_interval = params['dt_rng'], 
                                            per=params['per'],
                                            currency = params['fx'],
                                            tr_base = params['base'])
        if params['annual_factor'] != "":
            sortino = sortino.with_additional_parameters(ann_factor = params['annual_factor'])
        return sortino
        
    def __calc_tracking_error(self, **params):
        base = self.__get_base(**params)
        tracking_error = self.__bq.data.Tracking_Error(calc_interval = params['dt_rng'], 
                                                    per=params['per'],
                                                    currency = params['fx'],
                                                    tr_base = base,
                                                    benchmark_ticker = params['bench'])
        if params['annual_factor'] != "":
            tracking_error = tracking_error.with_additional_parameters(ann_factor = params['annual_factor'])            
        return tracking_error
    
    def __calc_alpha(self, **params):
        base = self.__get_base(**params)
        alpha = self.__bq.data.ALPHA(calc_interval = params['dt_rng'], 
                                  per=params['per'],
                                  currency = params['fx'],
                                  tr_base = base,
                                  benchmark_ticker = params['bench'], 
                                  annualized = params['annualize'])
        return alpha
            
    def __calc_beta(self, **params):
        base = self.__get_base(**params)
        beta = self.__bq.data.BETA(calc_interval = params['dt_rng'], 
                                 per = params['per'],
                                 currency = params['fx'],
                                 tr_base = base,
                                 benchmark_ticker = params['bench'])
        return beta
    
    def __calc_calmar_ratio(self, **params):
        base = self.__get_base(**params)
        calmar_ratio = self.__bq.data.calmar_ratio(calc_interval = params['dt_rng'], 
                                                     per = params['per'],
                                                     currency = params['fx'],
                                                     tr_base = base)
        return calmar_ratio
    
    def __calc_excess_return(self, **params):
        base = self.__get_base(**params)
        excess_return = self.__bq.data.excess_return(calc_interval = params['dt_rng'], 
                                             per = params['per'],
                                             currency = params['fx'],
                                             tr_base = base,
                                             benchmark_ticker = params['bench'], 
                                             annualized = params['annualize'])
        if params['annual_factor'] != "":
            excess_return = excess_return.with_additional_parameters(ann_factor = params['annual_factor'])  
        return excess_return
        
    def __calc_gain_to_loss_ratio(self, **params):
        base = self.__get_base(**params)
        gain_to_loss = self.__bq.data.gain_to_loss_ratio(calc_interval = params['dt_rng'], 
                                                     per = params['per'],
                                                     currency = params['fx'],
                                                     tr_base = base)
        return gain_to_loss
        
    def __calc_information_ratio(self, **params):
        base = self.__get_base(**params)
        info_ratio = self.__bq.data.information_ratio(calc_interval = params['dt_rng'], 
                                                  per = params['per'],
                                                  currency = params['fx'],
                                                  tr_base = base,
                                                  benchmark_ticker = params['bench'],
                                                  annualized = params['annualize'])        
        if params['annual_factor'] != "":
            info_ratio = info_ratio.with_additional_parameters(ann_factor = params['annual_factor'])  
        return info_ratio
        
    def __calc_jensen_alpha(self, **params):
        """ i have not incorporated ADJ_BETA_WEIGHT, and this will use the default value for ADJ_BETA_WEIGHT"""
        base = self.__get_base(**params)
        jensen_alpha = self.__bq.data.jensen_alpha(calc_interval = params['dt_rng'], 
                                                  per = params['per'],
                                                  currency = params['fx'],
                                                  tr_base = base,
                                                  benchmark_ticker = params['bench'],
                                                  risk_free_ticker = params['rf_id'])        
        if params['annual_factor'] != "":
            jensen_alpha = jensen_alpha.with_additional_parameters(ann_factor = params['annual_factor'])  
        return jensen_alpha    
    


In [6]:
class Gui():
    def __init__(self, bq):
        """
        # **Structure of App:
        
        # SlimAccordion (utility, accordion that folds into a button to take less space)
        #     AllFundFilters (reusable class that allows you to filter a fund)
        #         FundCalculations (class to contain all bql fund fields)
        #         FundFilters (class that implements the MultipleSelectMenu class for fund filter options)
        #             MultipleSelectMenu (class that takes a nested dictionary, and builds a ui to select options)
        #                 SettingsSummmary (part component of MultipleSelectMenu )
        #                 SelectionMenu (part component of MultipleSelectMenu )
        #                     NestedCheckBox (part component of MultipleSelectMenu )
        #                         Node (part component of MultipleSelectMenu )
        #                             SelectableOption (part component of MultipleSelectMenu )
        #                     ResultsPanel (part component of MultipleSelectMenu )
        #                         ResultLine (part component of MultipleSelectMenu )
        # TableWithBoxPlot (A datagrid, with a series of boxplots below aligned)
        #     CheckBoxSelection 
        # FundCharts (to chart selected funds)
        
        """
        importlib.reload(gui.fund_charts)
        importlib.reload(gui.table_boxplot)
        importlib.reload(gui.gui_utils_ryo)
        importlib.reload(gui.fund_filters)

        self.__funds_selected = None
        self.__all_funds = None
        self.__bq = bq
    
    def build(self):
        fund_calcs = FundCalculations(bq)
        self.__fund_gui = gui.fund_filters.AllFundFilters(self.__bq, fund_calcs, self.__main)
        self.__inputs = gui.gui_utils_ryo.SlimAccordion("Inputs:", self.__fund_gui)
        self.__main_gui = VBox()
        self.__pls_wait_msg = HTML(value="""<h3>Downloading...<i class="fa fa-spinner fa-spin fa-2x fa-fw" style="color:white;"></i></h3>""")
        self.__err_display = Output()
        self.__output_widget = gui.table_boxplot.TableWithBoxPlot()
        self.__hist_charts = gui.fund_charts.FundCharts(self.__bq, 
                                                        self.__fund_gui, 
                                                        self.__get_funds_univ, 
                                                        self.__err_display, 
                                                        self.__output_widget)
        # Layouts
        self.__main_gui.layout.overflow_x = 'visible'
        self.__main_gui.layout.overflow_y = 'visible'
        self.__main_gui.layout.border = '0.5px solid white'

        # pack
        self.__main_gui.children = [self.__inputs, self.__err_display]
        
        display(self.__main_gui)

    def __get_funds_univ(self):
        return self.__funds_selected, self.__all_funds
    
    def __tbl_cb(self, selected_rows, table):
        self.__funds_selected = table.iloc[selected_rows, [0]].values.tolist()
        self.__all_funds = table['ID'].tolist()

    def __main(self, tgt=None):
        self.__err_display.layout.height = '0px'
        self.__inputs.revert()
        with self.__err_display:
            try:
                clear_output()
                self.__main_gui.children = [self.__inputs, self.__pls_wait_msg]
                tbl = self.__fund_gui.get_data()
                self.__output_widget.update(tbl,
                                         ['ID', 'Name'], 
                                         [120, 220], 
                                         150, 
                                         self.__tbl_cb)
                self.__main_gui.children = [self.__inputs, self.__output_widget, self.__hist_charts]
            except:
                self.__err_display.layout.height = 'initial'
                self.__main_gui.children = [self.__inputs, self.__err_display]
                raise
    def get_err(self):
        return self.__err_display
fund_gui = Gui(bq)
fund_gui.build()